<a href="https://colab.research.google.com/github/Daarlens/ProBook/blob/main/NLP/NLP-2025/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_%E2%84%96_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🧪 **Лабораторный практикум № 4**  
# **Сравнительный анализ алгоритмов кластеризации текстовых данных на русскоязычных корпусах**

**Кафедра:** Кафедра анализа данных и технологий программирования  
**Дисциплина:** Обработка естественного языка  
**Уровень:** Магистратура, 2 курс  
**Преподаватель:** Арабов Муллошараф Курбонович  

---

## 🎯 1. Цели и задачи работы

### **Цель:**  
Получить комплексное практическое представление о **всех основных классических методах кластеризации текстов**, основанных на статистических и статических векторных представлениях, обученных на собственном корпусе. Научиться выбирать оптимальный алгоритм в зависимости от структуры данных, вычислительных ограничений и требований к интерпретируемости.

### **Задачи:**  
1. Использовать корпус из **Лабораторной работы №1** (≥10 000 документов).  
2. Применить **методы токенизации**, включая **собственную BPE-модель**, обученную в ЛР №1.  
3. Использовать **модели векторизации**, обученные в **Лабораторной работе №2**:  
   - Word2Vec (CBOW / Skip-Gram)  
   - FastText (CBOW / Skip-Gram)  
   - GloVe  
4. Реализовать и экспериментально сравнить **все основные классические методы кластеризации**:  
   - **Центроидные**: k-Means, Mini-Batch k-Means, Spherical k-Means  
   - **Плотностные**: DBSCAN, HDBSCAN  
   - **Иерархические**: агломеративная кластеризация (single, complete, average, ward)  
   - **Вероятностные**: Gaussian Mixture Models (GMM), Latent Dirichlet Allocation (LDA)  
   - **Графовые**: спектральная кластеризация  
5. Оценить качество с помощью **внутренних и внешних метрик**.  
6. Провести **визуализацию и интерпретацию** кластеров.  
7. Исследовать **устойчивость** к шуму, дисбалансу и объёму данных.  
8. Разработать **веб-интерфейс** для анализа результатов.

---

## 📚 2. Теоретические предпосылки

Кластеризация текстов — задача **обучения без учителя**, направленная на группировку документов по семантическому сходству **без разметки**. В данной работе рассматриваются **только классические, проверенные и интерпретируемые методы**, реализованные в зрелых библиотеках (`scikit-learn`, `Gensim`, `hdbscan`).

### **2.1. Токенизация (из ЛР №1)**
- Whitespace  
- Регулярные выражения  
- **Byte Pair Encoding (BPE)** — обученная на корпусе ЛР №1 модель (через `subword-nmt` или аналог)

### **2.2. Векторизация (из ЛР №2)**
- **TF-IDF**, **BM25** — статистические веса  
- **Word2Vec**, **GloVe**, **FastText** — статические эмбеддинги, обученные в ЛР №2 на том же корпусе  
> Вектор документа = среднее по токенам. Все модели — **собственные**, не предобученные извне.

### **2.3. Классические алгоритмы кластеризации**

Все методы реализованы в следующих библиотеках:

#### **Центроидные** (`scikit-learn`):
- `KMeans` — минимизирует сумму квадратов расстояний до центроидов  
- `MiniBatchKMeans` — масштабируемая версия для больших данных  
- `SphericalKMeans` — вариант для нормализованных (косинусных) пространств

#### **Плотностные** (`scikit-learn`, `hdbscan`):
- `DBSCAN` — обнаруживает кластеры произвольной формы, устойчив к выбросам  
- `HDBSCAN` — улучшенная иерархическая версия DBSCAN, **автоматически определяет число кластеров**

#### **Иерархические** (`scikit-learn`):
- `AgglomerativeClustering` с:
  - `linkage='ward'` (требует евклидовой метрики)  
  - `linkage='average'`, `'complete'`, `'single'` (работают с `metric='cosine'`)

#### **Вероятностные**:
- `GaussianMixture` (`scikit-learn`) — мягкое назначение точек кластерам  
- `LdaModel` (`Gensim`) — генеративная тематическая модель, рассматриваемая как кластеризатор по темам

#### **Графовые** (`scikit-learn`):
- `SpectralClustering` — кластеризация на основе собственных векторов матрицы схожести

> ⚠️ Все алгоритмы работают с `numpy.ndarray`, что делает их **универсальными** для любого векторного представления.

---

## 🧪 3. Методика эксперимента

### **3.1. Подготовка данных**
- Используйте корпус из ЛР №1.  
- Реализуйте `text_preprocessing.py` с поддержкой:
  - Очистки, лемматизации (`spaCy` или `pymorphy2`),  
  - Токенизации: whitespace, regex, **BPE (из ЛР №1)**.

> 💡 **spaCy** поддерживает 64-битные Python 3.7+ на Linux, macOS, Windows. Для лемматизации русского языка может потребоваться внешний морфологический анализатор.

### **3.2. Векторизация**
- Реализуйте `text_to_vector.py` с:
  - `TfidfVectorizer` (scikit-learn)  
  - `BM25Okapi` (rank-bm25)  
  - Загрузкой моделей из ЛР №2: `Word2Vec`, `FastText`, `GloVe` (через `Gensim`)  
- Нормализуйте векторы (`L2`) при использовании cosine similarity.

> 💡 **Gensim** — сверхбыстрая библиотека для эмбеддингов (C-ядра, параллелизм, streaming). Поддерживает Python 3.8+ и работает на всех платформах.

### **3.3. Кластеризация**
Реализуйте `clustering.py`, включающий **все перечисленные выше методы**.  
Для каждого:
- Подберите гиперпараметры (`k`, `eps`, `min_samples`, `n_components` и др.)  
- Примените стратифицированную или репрезентативную выборку при оценке

### **3.4. Оценка качества**
- **Внутренние метрики**: Silhouette, Calinski-Harabasz, Davies-Bouldin  
- **Внешние метрики** (если есть тематическая разметка из ЛР №3): ARI, NMI, V-measure  
- Постройте графики зависимости метрик от параметров

### **3.5. Интерпретация**
- Для TF-IDF: топ-10 слов на кластер  
- Для эмбеддингов: ближайшие слова к центроиду (через `model.most_similar()`)  
- Визуализация: UMAP → 2D scatter plot с цветами кластеров

> 📌 **LIME и SHAP** могут применяться **только если кластеризация используется как признак в downstream-задаче** (например, кластер → признак для классификатора). В чистой кластеризации они **не применимы напрямую**, но могут использоваться для анализа косвенных эффектов.

### **3.6. AutoML для кластеризации?**
> ❗ **Важно**: **AutoML-фреймворки (Auto-sklearn, TPOT, H2O)** **не поддерживают задачи без учителя** (включая кластеризацию) в стандартной конфигурации.  
> Auto-sklearn поддерживает **только задачи с разметкой** (классификация, регрессия).  
> Следовательно, **AutoML в данной работе НЕ ИСПОЛЬЗУЕТСЯ**.

### **3.7. Веб-интерфейс**
- Используйте `Streamlit` или `Gradio` для:
  - выбора метода токенизации/векторизации/алгоритма,  
  - визуализации кластеров,  
  - просмотра содержимого кластеров.

---

## 🔍 4. Дополнительные задания

1. Сравнение влияния BPE (из ЛР №1) и whitespace на качество  
2. Использование LDA как признакового пространства для k-Means  
3. Граф кластеров: построение матрицы схожести → community detection  
4. Автоматическое определение числа кластеров (silhouette analysis, gap statistic)

---

## 📄 5. Требования к отчёту

- Укажите, что BPE и эмбеддинги — **собственные, обученные в ЛР №1 и №2**  
- Приведите **таблицу сравнения всех методов** по качеству и времени  
- Включите **визуализации** и **примеры документов** из кластеров  
- Ссылки на GitHub и веб-демо  
- Рефлексия:  
  - Какой метод кластеризации показал наилучшие результаты для русского языка?  
  - Как влияет выбор векторизации на форму кластеров?  
  - Почему AutoML не применим к кластеризации?

---

## 📖 6. Используемые библиотеки

| Библиотека | Назначение | Примечание |
|-----------|-----------|-----------|
| `Gensim` | Word2Vec, FastText, GloVe, LDA | Супербыстрая, streaming, кроссплатформенная |
| `scikit-learn` | TF-IDF, k-Means, DBSCAN, GMM, спектральная кластеризация | Стандарт де-факто |
| `hdbscan` | HDBSCAN | Для кластеров произвольной формы |
| `spaCy` | Лемматизация, базовая токенизация | Требует установки языковых моделей |
| `subword-nmt` | BPE | Для обучения и применения BPE |
| `umap-learn` | Визуализация | Альтернатива t-SNE |
| `Streamlit` / `Gradio` | Веб-интерфейс | Быстрая разработка |

